## The Original EigenDecompression.EigenDecompose

See

* https://mobile.twitter.com/realize_ss/status/1615160291108745216
* https://qiita.com/lelele/items/8408410a94f5c6b8f76e

In [1]:
using LinearAlgebra
M = rand(100,100)#対角化したい行列
E, P = eigen(M)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
100-element Vector{ComplexF64}:
 -2.5334836589158582 - 0.8478529206464448im
 -2.5334836589158582 + 0.8478529206464448im
 -2.4035518173686343 - 0.7817859409132811im
 -2.4035518173686343 + 0.7817859409132811im
  -2.399698216509842 - 0.07762893715232995im
  -2.399698216509842 + 0.07762893715232995im
 -2.1959629487152927 - 1.5197368448980213im
 -2.1959629487152927 + 1.5197368448980213im
  -2.110125233357351 - 1.72909903986817im
  -2.110125233357351 + 1.72909903986817im
 -2.0400013792793787 - 0.9145479881653186im
 -2.0400013792793787 + 0.9145479881653186im
 -1.7802824316325463 - 0.2895732020926112im
                     ⋮
  1.8758015224197302 - 0.6591886088184818im
  1.8758015224197302 + 0.6591886088184818im
   2.123450548941241 - 1.3385422330742027im
   2.123450548941241 + 1.3385422330742027im
  2.4992808717651966 - 1.663405719109495im
  2.4992808717651966 + 1.663405719109495im
  2.5046424321436436 + 0.0im
   2.6

In [2]:
exp(eigen(M))

LoadError: MethodError: no method matching exp(::Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}})

[0mClosest candidates are:
[0m  exp([91m::Union{Float16, Float32, Float64}[39m)
[0m[90m   @[39m [90mBase[39m [90mspecial\[39m[90m[4mexp.jl:325[24m[39m
[0m  exp([91m::Adjoint{T, <:AbstractMatrix} where T[39m)
[0m[90m   @[39m [35mLinearAlgebra[39m [90mD:\Julia-1.9.0-beta2\share\julia\stdlib\v1.9\LinearAlgebra\src\[39m[90m[4mdense.jl:595[24m[39m
[0m  exp([91m::Transpose{T, <:AbstractMatrix} where T[39m)
[0m[90m   @[39m [35mLinearAlgebra[39m [90mD:\Julia-1.9.0-beta2\share\julia\stdlib\v1.9\LinearAlgebra\src\[39m[90m[4mdense.jl:596[24m[39m
[0m  ...


In [3]:
module EigenDecompression

export EigenDecompose, eigDecomp
using LinearAlgebra
import Base.*, Base./

#対角化された行列型
struct EigenDecompose{T<:Number} <: AbstractMatrix{T}
    P::AbstractMatrix{T}
    D::Diagonal{T}
    invP::AbstractMatrix{T}
end

#普通のMatrixを対角化する
function eigDecomp(mat::AbstractMatrix)
    E, P = eigen(mat)
    EigenDecompose(P, Diagonal(E), inv(P))
end

#EigenDecompose型に対する関数
Base.exp(eig::EigenDecompose) = EigenDecompose(eig.P, exp(eig.D), eig.invP)
*(eig::EigenDecompose, vec::AbstractVector) = eig.P * eig.D * eig.invP * vec
*(eig::EigenDecompose, sc::Number) = EigenDecompose(eig.P, eig.D*sc, eig.invP)
/(eig::EigenDecompose, sc::Number) = EigenDecompose(eig.P, eig.D/sc, eig.invP)

#普通のMatrixに戻す
Base.Array(eig::EigenDecompose) = eig.P * eig.D * eig.invP

end

Main.EigenDecompression

In [4]:
using .EigenDecompression

M = rand(100, 100)
eM = eigDecomp(M)
for i in 1:100
    v = rand(100)
    rnd = rand()
    @assert exp(M*rnd)*v ≈ exp(eM*rnd)*v
end

In [5]:
using BenchmarkTools

M = rand(100, 100);

#普通な方
function bench1(M)
    for i in 1:100
        v = rand(100)
        exp(M*rand())*v
    end
end

#今回実装した方
function bench2(M)
    eM = eigDecomp(M)
    for i in 1:100
        v = rand(100)
        exp(eM*rand())*v
    end
end

bench2 (generic function with 1 method)

In [6]:
@benchmark bench1(M)

BenchmarkTools.Trial: 67 samples with 1 evaluation.
 Range (min … max):  66.753 ms … 91.567 ms  ┊ GC (min … max): 2.45% … 5.47%
 Time  (median):     74.287 ms              ┊ GC (median):    2.62%
 Time  (mean ± σ):   75.154 ms ±  4.847 ms  ┊ GC (mean ± σ):  3.47% ± 1.35%

          ▁  █  ▁   ▁▆▁    ▃                                   
  ▄▁▁▄▁▁▇▇█▄▁█▁▇█▇▇▇███▄▇▄▇█▄▇▄▄▁▇▁▇▁▁▄▄▄▇▁▄▁▁▁▁▁▁▄▄▁▁▁▁▁▁▁▁▄ ▁
  66.8 ms         Histogram: frequency by time        89.1 ms <

 Memory estimate: 53.80 MiB, allocs estimate: 1900.

In [7]:
@benchmark bench2(M)

BenchmarkTools.Trial: 761 samples with 1 evaluation.
 Range (min … max):  5.921 ms …   9.813 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     6.511 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.565 ms ± 234.457 μs  ┊ GC (mean ± σ):  0.41% ± 1.92%

                         ▆█▆▄                                  
  ▂▁▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▂▂▃▅█████▇▆▄▃▃▄▃▂▃▂▂▂▃▁▂▂▂▂▂▁▂▂▃▂▂▂▂▂▂▂▂▂ ▃
  5.92 ms         Histogram: frequency by time         7.3 ms <

 Memory estimate: 1.82 MiB, allocs estimate: 1728.

## EigenDecomposedMatrices.EigenDecomposed

In [8]:
using LinearAlgebra
using BenchmarkTools

In [9]:
module EigenDecomposedMatrices

export EigenDecomposed

using LinearAlgebra
using Memoization

struct EigenDecomposed{
        T,
        TE<:AbstractVector,
        TP<:AbstractMatrix,
        TinvP<:AbstractMatrix
    } <: AbstractMatrix{T}
    E::TE
    P::TP
    invP::TinvP
end

function EigenDecomposed(E::AbstractVector, P::AbstractMatrix, invP::AbstractMatrix)
    EigenDecomposed{eltype(P), typeof(E), typeof(P), typeof(invP)}(E, P, invP)
end

function EigenDecomposed(A::AbstractMatrix)
    E, P = eigen(A)
    invP = ishermitian(A) ? P' : inv(P)
    EigenDecomposed(E, P, invP)
end

LinearAlgebra.eigvals(ed::EigenDecomposed) = ed.E
LinearAlgebra.eigvecs(ed::EigenDecomposed) = ed.P
inveigvecs(ed::EigenDecomposed) = ed.invP
@memoize Base.parent(ed::EigenDecomposed) = eigvecs(ed) * Diagonal(eigvals(ed)) * inveigvecs(ed)
Base.convert(::Type{Array}, ed::EigenDecomposed) = convert(Array, parent(ed))
for op in (:eltype, :size)
    @eval Base.$op(ed::EigenDecomposed) = $op(eigvecs(ed))
end
Base.getindex(ed::EigenDecomposed, I...) = getindex(parent(ed), I...)

Base.:*(c::Number, ed::EigenDecomposed) = EigenDecomposed(c*eigvals(ed), eigvecs(ed), inveigvecs(ed))
Base.:*(ed::EigenDecomposed, c::Number) = EigenDecomposed(eigvals(ed)*c, eigvecs(ed), inveigvecs(ed))
Base.:\(c::Number, ed::EigenDecomposed) = EigenDecomposed(c\eigvals(ed), eigvecs(ed), inveigvecs(ed))
Base.:/(ed::EigenDecomposed, c::Number) = EigenDecomposed(eigvals(ed)/c, eigvecs(ed), inveigvecs(ed))
for T in (AbstractVector, AbstractMatrix)
    @eval function Base.:*(ed::EigenDecomposed, v::$T)
        E, P, invP = eigvals(ed), eigvecs(ed), inveigvecs(ed)
        P * (Diagonal(E) * (invP * v))
    end
end

function exp_old(ed::EigenDecomposed)
    E, P, invP = eigvals(ed), eigvecs(ed), inveigvecs(ed)
    expE = exp.(E)
    expA = P * Diagonal(expE) * invP 
    EigenDecomposed(expE, P, invP)
end

LinearAlgebra.lmul!(c::Number, ed::EigenDecomposed) = lmul!(c, eigvals(ed))
LinearAlgebra.rmul!(ed::EigenDecomposed, c::Number) = rmul!(eigvals(ed), c)
LinearAlgebra.ldiv!(c::Number, ed::EigenDecomposed) = ldiv!(c, eigvals(ed))
LinearAlgebra.rdiv!(ed::EigenDecomposed, c::Number) = rdiv!(eigvals(ed), c)

for op in (:exp, :log, :sin, :cos)
    opE = Symbol(op, "E")
    op_eigendecomposed = Symbol(op, "_eigendecomposed")
    op_eigendecomposed! = Symbol(op_eigendecomposed, "!")
    op_eigendecomposed!_doc =
        """
        $op_eigendecomposed!(Y, ed::EigenDecomposed, $opE=similar(ed.E), tmpY=similar(Y))

        returns the `$op` of `ed` and stores the result in `Y`, overwriting the existing value of `Y`. 
        It does not overwrite `ed` and uses `$opE` and `tmpY` as workspaces.
        """
    @eval begin
        @doc $op_eigendecomposed!_doc
        function $op_eigendecomposed!(Y, ed::EigenDecomposed, $opE=similar(ed.E), tmpY=similar(Y))
            E, P, invP = eigvals(ed), eigvecs(ed), inveigvecs(ed)
            @. $opE = $op.(E)
            mul!(tmpY, P, Diagonal($opE))
            mul!(Y, tmpY, invP)
        end
        $op_eigendecomposed(ed::EigenDecomposed) = $op_eigendecomposed!(similar(eigvecs(ed)), ed)
        Base.$op(ed::EigenDecomposed) = $op_eigendecomposed(ed)
    end
end

end

Main.EigenDecomposedMatrices

In [10]:
?EigenDecomposedMatrices.exp_eigendecomposed!

exp_eigendecomposed!(Y, ed::EigenDecomposed, expE=similar(ed.E), tmpY=similar(Y))

returns the `exp` of `ed` and stores the result in `Y`, overwriting the existing value of `Y`.  It does not overwrite `ed` and uses `expE` and `tmpY` as workspaces.


In [11]:
?EigenDecomposedMatrices.log_eigendecomposed!

log_eigendecomposed!(Y, ed::EigenDecomposed, logE=similar(ed.E), tmpY=similar(Y))

returns the `log` of `ed` and stores the result in `Y`, overwriting the existing value of `Y`.  It does not overwrite `ed` and uses `logE` and `tmpY` as workspaces.


In [12]:
methods(EigenDecomposedMatrices.EigenDecomposed)

# 2 methods for type constructor:
 [1] Main.EigenDecomposedMatrices.EigenDecomposed(E::AbstractVector, P::AbstractMatrix, invP::AbstractMatrix)
     @ In[9]:19
 [2] Main.EigenDecomposedMatrices.EigenDecomposed(A::AbstractMatrix)
     @ In[9]:23

In [13]:
methodswith(EigenDecomposedMatrices.EigenDecomposed)

[1] *(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:39
[2] *(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[9]:40
[3] *(ed::Main.EigenDecomposedMatrices.EigenDecomposed, v::AbstractVector) @ Main.EigenDecomposedMatrices In[9]:44
[4] *(ed::Main.EigenDecomposedMatrices.EigenDecomposed, v::AbstractMatrix) @ Main.EigenDecomposedMatrices In[9]:44
[5] /(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[9]:42
[6] \(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:41
[7] convert(::Type{Array}, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:33
[8] cos(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:82
[9] eltype(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:35
[10] exp(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:82
[11] getindex(ed::Main.EigenDecomposedMatrices.EigenDecomposed, I...) @ Main.EigenDecomposedMatrices In[9]:37
[12] log(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:82
[13] parent(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices D:\.julia\packages\Memoization\ut5GT\src\Memoization.jl:162
[14] sin(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:82
[15] size(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:35
[16] eigvals(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:29
[17] eigvecs(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:30
[18] ldiv!(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:59
[19] lmul!(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:57
[20] rdiv!(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[9]:60
[21] rmul!(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[9]:58

In [14]:
methods(EigenDecomposedMatrices.exp_eigendecomposed!)

# 3 methods for generic function "exp_eigendecomposed!" from Main.EigenDecomposedMatrices:
 [1] exp_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed)
     @ In[9]:75
 [2] exp_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, expE)
     @ In[9]:75
 [3] exp_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, expE, tmpY)
     @ In[9]:75

In [15]:
methods(EigenDecomposedMatrices.log_eigendecomposed!)

# 3 methods for generic function "log_eigendecomposed!" from Main.EigenDecomposedMatrices:
 [1] log_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed)
     @ In[9]:75
 [2] log_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, logE)
     @ In[9]:75
 [3] log_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, logE, tmpY)
     @ In[9]:75

In [16]:
A = [
    2 -1 0
    -1 2 -1
    0 -1 2
]

edA = EigenDecomposedMatrices.EigenDecomposed(A)

3×3 Main.EigenDecomposedMatrices.EigenDecomposed{Float64, Vector{Float64}, Matrix{Float64}, Adjoint{Float64, Matrix{Float64}}}:
  2.0          -1.0  -3.33067e-16
 -1.0           2.0  -1.0
 -3.33067e-16  -1.0   2.0

In [17]:
log(edA)

3×3 Matrix{Float64}:
  0.51986   -0.623225  -0.173287
 -0.623225   0.346574  -0.623225
 -0.173287  -0.623225   0.51986

In [18]:
log(A)

3×3 Matrix{Float64}:
  0.51986   -0.623225  -0.173287
 -0.623225   0.346574  -0.623225
 -0.173287  -0.623225   0.51986

In [19]:
log(edA) ≈ log(A)

true

In [20]:
n = 2^8
M = 5I + randn(n, n)
v = randn(n)
c = randn()

edM = EigenDecomposedMatrices.EigenDecomposed(M)

Y = similar(eigvecs(edM))
expE = similar(eigvals(edM))
tmpY = similar(Y)

y = similar(eigvals(edM))
alpha = randn()
beta = randn();

In [21]:
edM

256×256 Main.EigenDecomposedMatrices.EigenDecomposed{ComplexF64, Vector{ComplexF64}, Matrix{ComplexF64}, Matrix{ComplexF64}}:
    6.10256-1.39336e-13im  …    0.645699-5.68473e-14im
  -0.136889-4.42578e-13im        1.19962-5.74855e-14im
   -1.32937-4.32892e-14im      -0.219375-3.0687e-14im
    1.03763+3.1694e-15im      -0.0427012+1.41823e-14im
  -0.770308+3.46176e-13im       0.190198-4.87065e-14im
   -1.27028-3.32044e-13im  …     1.39371-1.19868e-13im
   0.083904+1.28786e-13im      -0.286302+3.3039e-14im
  0.0414365+1.35275e-13im       -0.33207-1.09655e-13im
   0.760496-1.40848e-13im        1.44495+1.61796e-13im
   0.860957+1.97296e-14im       -2.40265+7.75963e-15im
  -0.190956-6.83334e-14im  …     1.34393-7.92604e-14im
 -0.0794981-3.68705e-13im        1.28217-7.96849e-14im
   -1.63634-4.98975e-13im      0.0599136+6.13194e-14im
           ⋮               ⋱            ⋮
   0.736175-5.13479e-13im      -0.106461-1.49797e-13im
    1.26557-3.94208e-14im  …    0.353323-1.60389e-13im
    1.106

In [22]:
dump(edM)

Main.EigenDecomposedMatrices.EigenDecomposed{ComplexF64, Vector{ComplexF64}, Matrix{ComplexF64}, Matrix{ComplexF64}}
  E: Array{ComplexF64}((256,)) ComplexF64[-11.185928886895997 + 0.0im, -9.918003076657737 - 2.969380460056924im, -9.918003076657737 + 2.969380460056924im, -9.419475563729012 - 1.798373714261982im, -9.419475563729012 + 1.798373714261982im, -8.695987083011541 - 4.593590578558174im, -8.695987083011541 + 4.593590578558174im, -8.246626677465969 - 4.599468626164498im, -8.246626677465969 + 4.599468626164498im, -8.045959472819053 - 1.0279037541349776im  …  19.010560732836613 + 2.7929130089091623im, 19.218667827137732 - 7.537136127564897im, 19.218667827137732 + 7.537136127564897im, 19.228255042870675 + 0.0im, 19.515907937682133 - 4.8973984082609165im, 19.515907937682133 + 4.8973984082609165im, 19.82873665912379 - 1.6123979833055297im, 19.82873665912379 + 1.6123979833055297im, 20.294741618502517 - 2.513863879404347im, 20.294741618502517 + 2.513863879404347im]
  P: Array{ComplexF64

In [23]:
M ≈ parent(edM) == Matrix(edM)

true

In [24]:
M ≈ edM

true

In [25]:
c*M ≈ c*edM ≈ edM*c

true

In [26]:
c\M ≈ c\edM ≈ edM/c

true

In [27]:
(
    exp(M)
    ≈ exp(edM)
    ≈ EigenDecomposedMatrices.exp_old(edM)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM, expE, tmpY)
)

true

In [28]:
@show typeof(y)
(
    exp(M) * v
    ≈ exp(edM) * v
    ≈ EigenDecomposedMatrices.exp_old(edM) * v
    ≈ mul!(y, EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM), v)
    ≈ mul!(y, EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM, expE, tmpY), v)
)

typeof(y) = Vector{ComplexF64}


true

In [29]:
@btime edM = EigenDecomposedMatrices.EigenDecomposed(M);

  38.121 ms (26 allocations: 3.53 MiB)


In [30]:
@btime exp($M) * $v
@btime exp($edM) * $v
@btime EigenDecomposedMatrices.exp_old($edM) * $v
@btime mul!($y, EigenDecomposedMatrices.exp_eigendecomposed!($Y, $edM), $v)
@btime mul!($y, EigenDecomposedMatrices.exp_eigendecomposed!($Y, $edM, $expE, $tmpY), $v);

  9.389 ms (16 allocations: 3.01 MiB)
  2.482 ms (7 allocations: 2.01 MiB)
  2.472 ms (9 allocations: 2.02 MiB)
  2.392 ms (3 allocations: 1.00 MiB)
  2.191 ms (0 allocations: 0 bytes)


In [31]:
n2 = 2^8
M2 = Symmetric(5I + randn(n2, n2))
v2 = randn(n)
c2 = randn()

edM2 = EigenDecomposedMatrices.EigenDecomposed(M2)

Y2 = similar(eigvecs(edM2))
expE2 = similar(eigvals(edM2))
tmpY2 = similar(Y2)

y2 = similar(eigvals(edM2))
alpha2 = randn()
beta2 = randn();

In [32]:
edM2

256×256 Main.EigenDecomposedMatrices.EigenDecomposed{Float64, Vector{Float64}, Matrix{Float64}, Adjoint{Float64, Matrix{Float64}}}:
  5.7688     -0.210314    -1.04732    …   0.330716   -1.91241    -0.445545
 -0.210314    4.89273      0.714977       1.24828    -0.243345    0.438352
 -1.04732     0.714977     4.84306       -1.84171    -0.679513    0.664189
 -0.125414   -0.915463     0.716194       0.183136    0.0994011  -0.40252
 -0.0418926  -0.0351171   -1.92683       -1.63571     0.751768    0.584647
  0.543023    2.47309     -0.345975   …  -1.19764    -1.48419     0.948547
 -1.50622     0.449476     0.698776      -0.981233    0.210106    1.23595
 -1.59532     0.214664    -1.27762       -0.671937   -0.0625707  -0.446278
 -0.818415   -0.206935     0.352311       0.78251     0.503864    0.722638
  1.15167     0.441184     0.181033      -0.903007   -0.0938306  -0.41866
 -0.135136   -1.53421      0.0989861  …  -0.204541   -0.248368   -1.21247
 -0.50282    -0.0313344   -0.547949       0.037

In [33]:
dump(edM2)

Main.EigenDecomposedMatrices.EigenDecomposed{Float64, Vector{Float64}, Matrix{Float64}, Adjoint{Float64, Matrix{Float64}}}
  E: Array{Float64}((256,)) [-26.131715086802494, -25.26541845689984, -25.130124929115535, -24.647211408674664, -23.838236745430642, -23.589557442307456, -23.52755272325446, -23.00410884553764, -22.373401296615597, -22.10887572421121  …  32.31397901209803, 32.61796226146069, 33.00229304584991, 33.34647573490567, 33.90093830493025, 34.19782406379767, 34.45131296780528, 34.999099046631756, 35.97041556516376, 36.64428414197512]
  P: Array{Float64}((256, 256)) [0.054136408500694407 -0.07154274126676044 … 0.051813129706927884 0.10922995237875112; 0.06946447463826523 -0.012073572503116959 … 0.01712012556690449 -0.07979224494521203; … ; -0.05846191789289258 0.07387667770151288 … -0.038175529854931306 0.014422950863896; -0.060576741303121615 0.053468174865639075 … 0.02462442156092207 -0.12553091238206668]
  invP: Adjoint{Float64, Matrix{Float64}}
    parent: Array{Float64}

In [34]:
M2 == parent(edM2) == Matrix(edM2)

false

In [35]:
M2 ≈ edM2

true

In [36]:
c2*M2 ≈ c2*edM2 ≈ edM2*c2

true

In [37]:
c2\M2 ≈ c2\edM2 ≈ edM2/c2

true

In [38]:
(
    exp(M2)
    ≈ exp(edM2)
    ≈ EigenDecomposedMatrices.exp_old(edM2)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2, expE2, tmpY2)
)

true

In [39]:
@show typeof(y2)
(
    exp(M2) * v2
    ≈ exp(edM2) * v2
    ≈ EigenDecomposedMatrices.exp_old(edM2) * v2
    ≈ mul!(y2, EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2), v2)
    ≈ mul!(y2, EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2, expE2, tmpY2), v2)
)

typeof(y2) = Vector{Float64}


true

In [40]:
@btime edM2 = EigenDecomposedMatrices.EigenDecomposed(M2);

  6.162 ms (14 allocations: 1.59 MiB)


In [41]:
@btime exp($M2) * $v2
@btime exp($edM2) * $v2
@btime EigenDecomposedMatrices.exp_old($edM2) * $v2
@btime mul!($y2, EigenDecomposedMatrices.exp_eigendecomposed!($Y2, $edM2), $v2)
@btime mul!($y2, EigenDecomposedMatrices.exp_eigendecomposed!($Y2, $edM2, $expE2, $tmpY2), $v2);

  6.864 ms (19 allocations: 2.60 MiB)
  663.800 μs (6 allocations: 1.00 MiB)
  678.200 μs (8 allocations: 1.01 MiB)
  642.600 μs (3 allocations: 514.17 KiB)
  617.900 μs (0 allocations: 0 bytes)
